Implement Word2Vec algorithm to compute vector representations of words. This example is using a small chunk of Wikipedia articles to train from.

In [9]:
import collections
import os
import random
import urllib
import zipfile

import numpy as np
import tensorflow as tf
os.chdir('/Users/chenggang/Documents')

In [56]:
# Training Parameters
learning_rate = 0.1
batch_size = 128
num_steps = 3000000
display_step = 10000
eval_step = 200000

# Evaluation Parameters
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

# Word2Vec Parameters
embedding_size = 200 # Dimension of the embedding vector
max_vocabulary_size = 50000# Total number of different words in the vocabulary
min_occurrence = 10# Remove all words that does not appears at least n times
skip_window = 3 # How many words to consider left and right
num_skips = 2# How many times to reuse an input to generate a label
num_sampled = 64 # Number of negative examples to sample

In [11]:
# # Download a small chunk of Wikipedia articles collection
# url = 'http://mattmahoney.net/dc/text8.zip'
# data_path = 'text8.zip'

# data_path = 'text8.zip'
# if not os.path.exists(data_path):
#     print("Downloading the dataset... (It may take some time)")
#     filename, _ = urllib.request.urlretrieve(url, data_path)
#     print("Done!")
    
# Unzip the dataset file. Text has already been processed
data_path= '/Users/chenggang/Documents/text8.zip'
with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()  

In [47]:
for i in range(len(text_words)):
    text_words[i] = text_words[i].decode('utf-8')

In [49]:
# Build the dictionary and replace rare words with UNK token
count = [('UNK',-1)]
# Retrieve the most common words 统计前max_vocabulary_size-1个
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
# Remove samples with less than 'min_occurrence' occurrences
for i in range(len(count) - 1, -1, -1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        # The collection is ordered, so stop when 'min_occurrence' is reached
        break
# Compute the vocabulary size
vocabulary_size = len(count)
# Assign an id to each word
word2id = dict()
for i, (word, _)in enumerate(count):
    word2id[word] = i

data = list()
unk_count = 0
for word in text_words:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in dictionary
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)
count[0] = ('UNK', unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

Words count: 17005207
Unique words: 253854
Vocabulary size: 47135
Most common words: [('UNK', 444176), ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]


In [50]:
data_index = 0
# Generate training batch for the skip-gram model
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # get window size (words left and right + current one)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [51]:
# Input data
X = tf.placeholder(tf.int32, shape=[None])
# Input label
Y = tf.placeholder(tf.int32, shape=[None, 1])

# Ensure the following ops & var are assigned on CPU
# (some ops are not compatible on GPU)
with tf.device('/cpu:0'):
    # Create the embedding variable (each row represent a word embedding vector)
    embedding = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    # Lookup the corresponding embedding vectors for each sample in X
    X_embed = tf.nn.embedding_lookup(embedding, X)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch
loss_op = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights,
                   biases=nce_biases,
                   labels=Y,
                   inputs=X_embed,
                   num_sampled=num_sampled,
                   num_classes=vocabulary_size))

# Define the optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluation
# Compute the cosine similarity between input data embedding and every embedding vectors
X_embed_norm = X_embed / tf.sqrt(tf.reduce_sum(tf.square(X_embed)))
embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
cosine_sim_op = tf.matmul(X_embed_norm, embedding_norm, transpose_b=True)

In [57]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Testing data
    x_test = np.array([word2id[w] for w in eval_words])

    average_loss = 0
    for step in range(1, num_steps + 1):
        # Get a new batch of data
        batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
        # Run training op
        _, loss = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
        average_loss += loss

        if step % display_step == 0 or step == 1:
            if step > 1:
                average_loss /= display_step
            print("Step " + str(step) + ", Average Loss= " + \
                  "{:.4f}".format(average_loss))
            average_loss = 0

        # Evaluation
        if step % eval_step == 0 or step == 1:
            print("Evaluation...")
            sim = sess.run(cosine_sim_op, feed_dict={X: x_test})
            for i in range(len(eval_words)):
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = '"%s" nearest neighbors:' % eval_words[i]
                for k in range(top_k):
                    log_str = '%s %s,' % (log_str, id2word[nearest[k]])
                print(log_str)

Step 1, Average Loss= 499.4827
Evaluation...
"five" nearest neighbors: cream, natively, tunneling, fares, prefers, arts, solute, staircase,
"of" nearest neighbors: sdtv, millennium, oxus, dogged, runyon, drop, vauxhall, ensues,
"going" nearest neighbors: surge, smartest, imposition, manhunter, donors, ackermann, imperium, optimization,
"hardware" nearest neighbors: wozniak, merger, dissected, preferences, langford, follower, bushmen, cheerleaders,
"american" nearest neighbors: nationality, inactive, waging, entire, stilicho, associating, compromised, camden,
"britain" nearest neighbors: healthier, compositional, burckhardt, zhdanov, marjoram, libertarian, buffy, dojo,
Step 10000, Average Loss= 199.5958
Step 20000, Average Loss= 93.7941
Step 30000, Average Loss= 65.1458
Step 40000, Average Loss= 50.9887
Step 50000, Average Loss= 41.3762
Step 60000, Average Loss= 36.2016
Step 70000, Average Loss= 31.9625
Step 80000, Average Loss= 29.6241
Step 90000, Average Loss= 27.2182
Step 100000, Ave

Step 1350000, Average Loss= 6.2179
Step 1360000, Average Loss= 6.2876
Step 1370000, Average Loss= 6.2409
Step 1380000, Average Loss= 6.2473
Step 1390000, Average Loss= 6.2600
Step 1400000, Average Loss= 6.2092
Evaluation...
"five" nearest neighbors: four, six, three, two, eight, seven, zero, one,
"of" nearest neighbors: the, and, including, its, became, from, with, through,
"going" nearest neighbors: took, to, he, were, their, only, today, having,
"hardware" nearest neighbors: local, instead, public, military, greek, based, strong, low,
"american" nearest neighbors: english, german, french, british, born, film, s, john,
"britain" nearest neighbors: europe, country, movement, society, local, along, countries, community,
Step 1410000, Average Loss= 6.2731
Step 1420000, Average Loss= 6.2860
Step 1430000, Average Loss= 6.2503
Step 1440000, Average Loss= 6.2734
Step 1450000, Average Loss= 6.1605
Step 1460000, Average Loss= 6.1820
Step 1470000, Average Loss= 6.2146
Step 1480000, Average Loss

Step 2640000, Average Loss= 5.5609
Step 2650000, Average Loss= 5.6353
Step 2660000, Average Loss= 5.7030
Step 2670000, Average Loss= 5.6370
Step 2680000, Average Loss= 5.5870
Step 2690000, Average Loss= 5.6784
Step 2700000, Average Loss= 5.6135
Step 2710000, Average Loss= 5.6581
Step 2720000, Average Loss= 5.6638
Step 2730000, Average Loss= 5.6027
Step 2740000, Average Loss= 5.6842
Step 2750000, Average Loss= 5.6816
Step 2760000, Average Loss= 5.6454
Step 2770000, Average Loss= 5.6876
Step 2780000, Average Loss= 5.6067
Step 2790000, Average Loss= 5.6246
Step 2800000, Average Loss= 5.6347
Evaluation...
"five" nearest neighbors: three, four, six, eight, seven, two, nine, zero,
"of" nearest neighbors: the, in, including, following, and, from, under, became,
"going" nearest neighbors: again, having, when, once, to, came, without, they,
"hardware" nearest neighbors: personal, private, software, system, program, public, future, systems,
"american" nearest neighbors: actor, singer, born, auth